Import all packages in a cell

In [1]:
# Import all packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Repeated functions
def getMode(df, attribute, condAtt, cond):
    return (df[attribute][df[condAtt]==cond].mode()[0])

def replaceWithMode(df, attribute, condAtt, cond):
    mode = getMode(df, attribute, condAtt, cond)
    df[attribute] = df[attribute].mask(((df[condAtt]==cond) & (df[attribute]=='?')), mode)
    

# read the dataset and set skipinitialspace to true to be able to .replace
df = pd.read_csv('./HouseholderAtRisk(1).csv', skipinitialspace=True)

# show all columns 
print(df.info())

# Task 1 question 1
# Show proportion of high risk
risks = df['AtRisk'].value_counts()
print("Proportion of high risk = " + str(risks[0]/len(df)))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39999 entries, 0 to 39998
Data columns (total 18 columns):
ID                        39999 non-null int64
Age                       39032 non-null float64
WorkClass                 39027 non-null object
Weighting                 38707 non-null float64
Education                 39027 non-null object
NumYearsEducation         39027 non-null float64
Marital-Status            39027 non-null object
Occupation                39013 non-null object
Relationship              39027 non-null object
Race                      45 non-null object
Gender                    39027 non-null object
CapitalLoss               39027 non-null float64
CapitalGain               39027 non-null float64
CapitalAvg                39027 non-null float64
NumWorkingHoursPerWeek    39027 non-null float64
Sex                       39027 non-null float64
CountryOfOrigin           39969 non-null object
AtRisk                    39999 non-null object
dtypes: float64(8), int

In [2]:
# Drop rows with multiple columns containing NaN values
df = df.dropna(subset=["Relationship", "Sex", "NumYearsEducation"], how='all')

# Replace inconsistency in CountryOfOrigin
df['CountryOfOrigin'] = df['CountryOfOrigin'].replace("US", "USA").replace("United-States", "USA")

# Removing Gender to use numerical binary for Sex where 0 = Male, 1 = Female
df.drop('Gender', axis=1, inplace=True)

# Replace -1 value in Age with mean value
meanAge = df['Age'].mean()
df.loc[df['Age']==-1, 'Age'] = meanAge

# Round off age
df['Age'] = df['Age'].astype(int)

# Drop race as there are 39954 NaN vs 45 labelled classes
df.drop('Race', axis=1, inplace=True)


# Sets upper boundary of 90 hours work week in NumWorkingHoursPerWeek and fills with mean value
meanWorkHours = df['NumWorkingHoursPerWeek'].mean()
df['NumWorkingHoursPerWeek'] = df['NumWorkingHoursPerWeek'].where(df['NumWorkingHoursPerWeek'] <= 90, meanWorkHours)  
# Round off hours                             
df['NumWorkingHoursPerWeek'] = df['NumWorkingHoursPerWeek'].astype(int)

# Replaces missing data with mean of column
df['Weighting'] = df['Weighting'].fillna(df['Weighting'].mean())


# Replaces missing data with unknown Occupation
df['Occupation'] = df['Occupation'].fillna("?")



# Replaces unknowns with the mode of attribute
# By WorkClass
replaceWithMode(df,'Occupation','WorkClass','Federal-gov')
replaceWithMode(df,'Occupation','WorkClass','Self-emp-inc')
replaceWithMode(df,'Occupation','WorkClass','Private')
replaceWithMode(df,'Occupation','WorkClass','Never-worked')

# By education
replaceWithMode(df, 'Occupation', 'Education', 'HS-grad')
replaceWithMode(df, 'WorkClass', 'Education', 'HS-grad')

replaceWithMode(df, 'Occupation', 'Education', 'Some-college')
replaceWithMode(df, 'WorkClass', 'Education', 'Some-college')


# Never-worked adults will have occupation set as other services as a generic unknown
df['Occupation'] = df['Occupation'].mask(((df['WorkClass']=='Never-worked') & (df['Occupation']=='?')), "Other-service")




In [18]:
#print(df['Occupation'].value_counts())
#print(df['WorkClass'].value_counts())


#print(df['Education'].unique())

#replaceWithMode(df, 'Occupation', 'Education', 'HS-grad')


print(df['WorkClass'][df['Education']=='Some-college'].value_counts())
print(df['Occupation'][df['Education']=='Some-college'].value_counts())





Private             6700
Self-emp-not-inc     585
Local-gov            484
State-gov            391
Federal-gov          277
Self-emp-inc         266
Never-worked           1
Name: WorkClass, dtype: int64
Adm-clerical         2185
Sales                1205
Exec-managerial      1019
Craft-repair          986
Other-service         952
Prof-specialty        512
Machine-op-inspct     400
Tech-support          337
Handlers-cleaners     318
Transport-moving      312
Protective-serv       245
Farming-fishing       207
Priv-house-serv        22
Armed-Forces            4
Name: Occupation, dtype: int64
